## Importing libraries

In [ ]:
import pandas as pd
import seaborn as sns
import missingno as msno
import plotly.express as px

In [ ]:
NA_THR = 0.5
COLOR_PALETTE = px.colors.qualitative.Set2
SNS_CMAP = sns.light_palette("orange", as_cmap=True)

## Data Understanding (Explanation of key variables)

1. id - Unique id for loan listing
2. member_id - Unique id for the borrower
3. loan_amnt - Amount requested by the borrower
4. funded_amnt - Amount appproved by the lending club
5. funded_amnt_inv - Amount lent by the investor or the final amount which the borrower received
6. term - Time for which the amount is lent
7. int_rate - Interest rate for the loan
8. installment - Amount to be paid every month
9. grade - Grade assigned by the lending club, parameter for determinig the risk , ranges from A-G
10. sub_grade - Subgrade assigned from 1-5 for all the grades. Like A1-A5, based on those the interest keeps on increasing as the risk also increases
11. emp_title - Job title for the borrower
12. emp_length - No of employment years
13. home_ownership -  Indicates if the borrower has own house, rented, mortagaged or other
14. verification_status - Indicates if income source is verified or not
15. annual_inc - Annual income of the borrower
16. loan_status - Indicates if the person has successfully paid a loan or is a defaulter
17. purpose - Indicates the purpose of the loan
18. title - Loan title provided by borrower , also describes the purpose
19. zip_code - Aip code of his communication residence
20. addr_state - Communication address
21. dti - A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
22. pub_rec_bankruptcies - Number of public record bankruptcies

##  Data cleaning

Following is the data cleaning process followed:
 - Remove rows where the borrower is still in the process of paying out the loan
 - Remove customer behaviour variables as those are not available at the time of loan application
 - Drop columns where there are more than 50% NA values
 - Drop columns where there is only one unique value as these will not contribute to the analysis
 - Change interest rate column from string to numeric
 - The target variable (Loan status) is binarized such that 1 represents default and 0 represents that the loan has been fully paid, for ease of analysis

In [ ]:
loan_df = pd.read_csv("loan.csv", low_memory=False)

## Remove records where loan has not been completed
loan_df = loan_df[loan_df.loan_status != "Current"]

## Remove customer behaviour variables
columns_to_drop = ["delinq_2yrs","earliest_cr_line","inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",
    "out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee",
    "recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","last_credit_pull_d","application_type"]

loan_df.drop(columns_to_drop, axis=1, inplace=True)

# Drop columns with more than 50% of NA values
loan_df = loan_df.dropna(axis='columns', thresh=NA_THR * loan_df.shape[0])


In [ ]:
##dropping columns with unique values as they won't contribute to the analysis
loan_df.nunique().sort_values()


In [ ]:
#dropping all columns with unique values
loan_df.drop(['tax_liens', 'delinq_amnt', 'chargeoff_within_12_mths', 'acc_now_delinq', 'policy_code', 'collections_12_mths_ex_med', 'initial_list_status', 'pymnt_plan'], axis=1, inplace=True)
loan_df.info()

In [ ]:
## Binarize y variable
loan_df["Defaulted"] = 0
loan_df.loc[loan_df.loan_status == "Charged Off", "Defaulted"] = 1


##Creating a new column for ratio of approved amount by investor, 
loan_df['approved_amount_ratio']= (loan_df.funded_amnt_inv/loan_df.loan_amnt) * 100

## Change interest rate column to numeric
loan_df["int_rate_numeric"] = loan_df.int_rate.apply(lambda x: x.strip("%")).astype(float)

## Remove rows where employee length is not available
loan_df = loan_df[~loan_df.emp_length.isna()]


In [ ]:
loan_df.loan_status.value_counts()

In [ ]:
msno.matrix(loan_df)

The graph above shows that the largest number of missing values are present in the 'desc' column folowed by the 'emp_title' column. Since these two variables will not be used for further analysis, the missing values are left as is. However, if we wanted to impute the missing values for the variable  'emp_title', we could use the mode of the column to do so

## Univariate Analysis

### Analysing the distribution of annual income

In [ ]:
px.violin(loan_df['annual_inc'],box=True, color_discrete_sequence=COLOR_PALETTE)

### Observation
It can be seen that the annual income variable is highly affected by outliers. Since an annual income of more than $200000 is unlikely, these outliers can be removed from the dataset as follows

In [ ]:
loan_df = loan_df[loan_df.annual_inc <=200000]

### Analysing the distribution of annual income post outlier removal

In [ ]:
### Analysing the distribution of annual income

In [ ]:
px.violin(loan_df, y="annual_inc",box=True, color_discrete_sequence=COLOR_PALETTE)

### Analysing the distribution of the loan_amnt,funded_Amnt and funded_amnt_inv

In [ ]:
amnt_types = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv']
px.violin(loan_df[amnt_types],box=True, color_discrete_sequence=COLOR_PALETTE)

### Observation:

1. Each of the variables loan_amnt, funded_amnt and funded_amnt_inv have very similar distributions
2. The range amnt for the loan_amnt and funded_amnt is 500-35k, which indicates all the loans are atleast approved by Lending club. For funded_amnt_inv ranges from 0-35k. Since the minimum value is 0, it means there are loans which were not approved by the investor

### Analysing the distribution of house_ownership variable

In [ ]:
px.histogram(loan_df['home_ownership'], color_discrete_sequence=COLOR_PALETTE)

### Observations
 Most no of borrowers are either having a rented or a mortagaged house, there are very less people with own house

### Analysing the distribution of the employee years of experience

In [ ]:
px.histogram(loan_df,x="emp_length",barmode="group",category_orders={"emp_length": ["< 1 year", "1 year", "2 years", "3 years", "4 years", "5 years", "6 years", "7 years", "8 years", "9 years", "10+ years"]}, color_discrete_sequence=COLOR_PALETTE)

### Observation:

Most borrowers have a work experience of more than 10 years

### Distribution of loan_status

In [ ]:
px.histogram(loan_df['loan_status'], color_discrete_sequence=COLOR_PALETTE)

### Observation

Most people have fully paid their loans i.e. around 32k, whereas around 5k have defaulted 

### Analysing the distribution of loan grade

In [ ]:
px.histogram(loan_df,x="grade",barmode="group",category_orders={"grade": ["A","B","C","D","E","F","G"]}, color_discrete_sequence=COLOR_PALETTE)

### Observation
The grades have a right-skewed distribution with most of the borrowers having a high grade assigned

### Analysing the dsitribution of verification_status


In [ ]:
px.histogram(loan_df,x="verification_status",barmode="group", color_discrete_sequence=COLOR_PALETTE)

### Analysing the dsitribution of pub_rec_derogatory

In [ ]:

px.histogram(loan_df,x="pub_rec_bankruptcies",histnorm='probability',barmode='group',category_orders={"pub_rec_bankruptcies": ["0","1","2"]}, color_discrete_sequence=COLOR_PALETTE)



### Observation
Almost 99% don't have a record of bankruptcy


### Analysing the dsitribution of issue_date

In [ ]:
loan_df['issue_d'] = pd.to_datetime(loan_df.issue_d, format='%b-%y')
#if there are any loan_df before unix timestamp they are converted to 21 century values, 
#so subtracting values more than 2020  with 100 will give more actual time
loan_df['issue_d'] = loan_df['issue_d'].apply(lambda x: x-pd.DateOffset(years=100) if x.year > 2020 else x)
loan_df['issue_d_year'] = loan_df.issue_d.dt.year
loan_df['issue_d_month'] = loan_df.issue_d.dt.strftime('%b')
loan_df['issue_d_weekday'] = loan_df.issue_d.dt.weekday
#loan_df type conversion of year and weekday
loan_df['issue_d_year'] = loan_df['issue_d_year'].astype(object)
loan_df['issue_d_weekday'] = loan_df['issue_d_weekday'].astype(object)

In [ ]:
px.histogram(loan_df,x="issue_d_year",histnorm='probability', color_discrete_sequence=COLOR_PALETTE)

In [ ]:
px.histogram(loan_df,x="issue_d_month",histnorm='probability', color_discrete_sequence=COLOR_PALETTE)

### Observations

1. Loans issued by lending club have doubled every year
2. At the end of the year i.e. Oct, Nov, Dec , most of the loans have been issued

### Analysing the dsitribution of loan term

In [ ]:
px.histogram(loan_df,x="term",histnorm='probability', color_discrete_sequence=COLOR_PALETTE)

### Observation

1. Most of the loans have been given for a term of 36 months

## Segmented Univariate analysis & Bivariate analysis


### Loan status vs Loan amount

In [ ]:
px.box(loan_df, x="loan_status", y="loan_amnt", color_discrete_sequence=COLOR_PALETTE)

### Observation
1. Median loan amounts do not vary greatly between the defaulters and non-defaulters. The 75th percentile values , however, are larger for defaulters as compared to non-defaulters


### Loan amount vs verified status

In [ ]:
px.box(loan_df, x="verification_status", y="loan_amnt", color_discrete_sequence=COLOR_PALETTE)

In [ ]:
px.box(loan_df, x="verification_status", y="loan_amnt",color='Defaulted', color_discrete_sequence=COLOR_PALETTE)

### Observation
1. Loan amount is higher is for verified sources as compared to source verified and not verified sources
2. Within each verification status, the loan amounts issued to defaulters and non-defaulters follow similar distributions

### Loan amount vs grade 

In [ ]:

px.box(loan_df, x="grade", y="loan_amnt",category_orders={"grade": ["A","B","C","D","E","F","G"]}, color_discrete_sequence=COLOR_PALETTE)

### Observation
1. As the loan amount increases, the loan gets riskier and hence the loan grade increases (alphabetically)

### Loan amount vs term

In [ ]:
px.box(loan_df, x="term", y="loan_amnt", color_discrete_sequence=COLOR_PALETTE)

### Observation
1. For higher loan amount, the tenure is high

### Loan amount vs home ownership

In [ ]:
px.box(loan_df, x="home_ownership", y="loan_amnt",color='Defaulted', color_discrete_sequence=COLOR_PALETTE)

### Observation
1. Defaulters who live in "mortgaged" houses or "other" form of housing have requested for higher loan amounts as compared to non-defaulters and the remaining housing categories

### Loan amount vs issued year and issued month

In [ ]:
px.box(loan_df, x="issue_d_year", y="loan_amnt", color_discrete_sequence=COLOR_PALETTE)

### Observation
1. The loan amounts requested by borrowers, on average, have increased over the years

In [ ]:
px.box(loan_df, x="issue_d_year", y="loan_amnt",color='Defaulted', color_discrete_sequence=COLOR_PALETTE)

In [ ]:
px.box(loan_df, x="issue_d_month", y="loan_amnt", color_discrete_sequence=COLOR_PALETTE)

In [ ]:
px.box(loan_df, x="issue_d_month", y="loan_amnt",color='Defaulted', color_discrete_sequence=COLOR_PALETTE)

### Observation
1. Except for the years 2009 & 2010, defaulters had, on average, requested for higher loan amounts than non-defaulters
2. Towards the second half of the year, defaulters had, on average, requested for higher loan amounts than non-defaulters

### Interest rate v/s loan tenure

In [ ]:
px.box(loan_df, x="term", y="int_rate_numeric", color_discrete_sequence=COLOR_PALETTE)

### Observation

1. For 60 months term , interest rate charged is higher

### Interest rate v/s Employee years of experience

In [ ]:
px.box(loan_df, x="emp_length", y="int_rate_numeric",category_orders={"emp_length": ["< 1 year", "1 year", "2 years", "3 years", "4 years", "5 years", "6 years", "7 years", "8 years", "9 years", "10+ years"],}, color_discrete_sequence=COLOR_PALETTE
    )

### Observation
1. We can see that there is not much differnce in interest rates across employee experience

### Interest rate v/s loan purpose

In [ ]:
px.box(loan_df, x="purpose", y="int_rate_numeric", color_discrete_sequence=COLOR_PALETTE, color="Defaulted")

### Observation
1. for small business ,debt_consolidation and house , loans are given at high interest

### Interest rate v/s verification status

In [ ]:
px.box(loan_df, x="verification_status", y="int_rate_numeric", color_discrete_sequence=COLOR_PALETTE)

### Interest rate v/s loan grade

In [ ]:
px.box(loan_df, x="grade", y="int_rate_numeric",category_orders={"grade": ["A","B","C","D","E","F","G"]}, color_discrete_sequence=COLOR_PALETTE)

### Interest rate v/s sub grade

In [ ]:
px.box(loan_df, x="sub_grade", y="int_rate_numeric",category_orders={"sub_grade": ["A1","A2","A3","A4","A5","B1","B2","B3","B4","B5","C1","C2","C3","C4","C5","D1","D2","D3","D4","D5","E1","E2","E3","E4","E5","F1","F2","F3","F4","F5","G1","G2","G3","G4","G5"]}, color_discrete_sequence=COLOR_PALETTE)

### Observation
1. Interest rate increases with decrease in grade

### How does employee years of experience affect probability of default?

In [ ]:
px.histogram(
    loan_df, x="emp_length", color="Defaulted", color_discrete_sequence=COLOR_PALETTE, 
    barmode="group", 
    category_orders={"emp_length": ["< 1 year", "1 year", "2 years", "3 years", "4 years", "5 years", "6 years", "7 years", "8 years", "9 years", "10+ years"]}
    )

In [ ]:
pd.crosstab(loan_df['emp_length'], loan_df['loan_status'],
            normalize='index').sort_index().round(2).style.background_gradient(cmap = SNS_CMAP)

### Observation
1. The probability of default is slightly higher for borrowers that have more than 10 years of experience as compared to the other levels

## How does annual income affect probability of default?

In [ ]:
px.violin(loan_df[loan_df.annual_inc <= 200000], x="annual_inc", color="Defaulted", color_discrete_sequence=COLOR_PALETTE, box=True)

### observation
1. Annual income is, on average, lower for defaulters as compared to non-defaulters


## How does loan grade affect probability of default?

In [ ]:
px.histogram(
    loan_df, x="grade", color="Defaulted", histfunc="count", barmode="group", 
    category_orders={"grade": ["A", "B", "C", "D", "E", "F", "G"]}, color_discrete_sequence=COLOR_PALETTE)

In [ ]:
pd.crosstab(loan_df['grade'], loan_df['loan_status'],
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

### observation
1. The proability of default is higher for low-grade loans than high-grade loans

## How does loan sub grade affect probability of default?

In [ ]:
px.histogram(
    loan_df, x="sub_grade", color="Defaulted", histfunc="count", barmode="group", 
    category_orders={"sub_grade": loan_df["sub_grade"].value_counts().index.sort_values()}, color_discrete_sequence=COLOR_PALETTE)

In [ ]:
pd.crosstab(loan_df['sub_grade'], loan_df['loan_status'],
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

### observation
1. The proability of default is higher for low subgrade loans than high subgrade loans

## How does interest rate affect probability of default?

In [ ]:
px.violin(loan_df, x="int_rate_numeric", color="Defaulted", color_discrete_sequence=COLOR_PALETTE, box=True)

## Observation

1. People who have defaulted have higher interest rates on their loans as compared to those that haven't

## How does loan term affect probability of default?

In [ ]:
px.histogram(
    loan_df, x="term", color="Defaulted", color_discrete_sequence=COLOR_PALETTE, 
    barmode="group")

In [ ]:
pd.crosstab(loan_df['term'], loan_df['loan_status'],
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

## Observation

1. People who have defaulted have higher loan tenures have a higher probability of defaulting than those who don't

## How does Debt to Income ratio impact default probability?

In [ ]:
px.violin(loan_df, x="dti", color="Defaulted", color_discrete_sequence=COLOR_PALETTE, box=True)

### Observation
1. As should be expected, debt to income ratio is higher for defaulters

## How does home ownership affect default probability?

In [ ]:
px.histogram(
    loan_df[loan_df.home_ownership != "NONE"], x="home_ownership", color="Defaulted", histfunc="count", barmode="group", color_discrete_sequence=COLOR_PALETTE)

In [ ]:
df = loan_df[loan_df.home_ownership != "NONE"]
pd.crosstab(df['home_ownership'], df['loan_status'],
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

### Observation
1. Home ownership alone does not seem to have a huge effect on default probability. While it seems like the default probability is higher for those who have "other" home ownership, the sample size is too small to make such a claim here

## How does verification status affect default probability?

In [ ]:
pd.crosstab(loan_df["verification_status"], loan_df["loan_status"], 
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

### Observation
1. Verified loans seem to have a higher default probability than not verified or source verified loans

## How does loan purpose affect default probability?

In [ ]:
pd.crosstab(loan_df["purpose"], loan_df['loan_status'], 
                   normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

### Observation :
    1. For small business, defaulters are significantly higher. This could be due to the high interest rates charged to small business owners.

### How does pub_rec_bankruptcies affect default probability

In [ ]:
pd.crosstab(loan_df["pub_rec_bankruptcies"], loan_df["loan_status"],
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)

### Observation :
1. As the public record for bankrutupcy increases, default rate increase

### How does default ratio vary for different interest rate groups? (Type-driven metric)

In [ ]:
loan_df['int_rate_group'] = pd.cut(loan_df['int_rate_numeric'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])

pd.crosstab(loan_df["int_rate_group"], loan_df["loan_status"],
            normalize='index').round(2).style.background_gradient(cmap = SNS_CMAP)


## Conclusion from univariate and univariate segmented analysis


 1. As the interest rate increase, no. of defaulters increases
 2. As record of pub_rec_bankruptcies increase, no. of defaulters increases
 3. People taking loans for a purpose of small business have highest rate of defaulters, as their interest rate is higher
 4. People who have their incomes verified by LC are the highest defaulters and also their interest rates are higher
 5. For home_ownership, we don't see any significant difference, though people with "other" has the highest ratio
 6. As loan grades and subgrades deteriorate, the probability of default increases
 7. As debt to income ratio increases, rate of default also increases
 8. As annual income increases, probability of default reduces
 9. As loan tenure increases, probability of default increases
 10. Employee years of experience does not have a great impact on default probability 